In [11]:

# %%
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_scatter import scatter_add, scatter_max
from pytorch_wavelets import DWT1DForward, DWT1DInverse
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")
import numpy as np
import os
import matplotlib.pyplot as plt
from typing import Optional
from torch_geometric.utils import to_dense_adj, dense_to_sparse
from torch.nn import Sequential, LayerNorm, ReLU
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import math
import pandas as pd

from torch.nn import Linear, Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax
from torch.utils.data import random_split
from torch.nn import InstanceNorm1d
from torch import Tensor

import optuna

from RevIN import RevIN

    
class Dataset_Custom(Dataset):
    def __init__(self, root_path, flag='train', size=None,
                 features='M', data_path='ETTh1.csv',
                 target='OT', scale=True, timeenc=0, freq='h',step_size=1):
        # Initialize with seq_len and pred_len only
        if size is None:
            self.seq_len = 96  # Default value, can be adjusted
            self.pred_len = 96  # Default value, can be adjusted
        else:
            self.seq_len, self.pred_len = size[:2]
            
        
        
        assert flag in ['train', 'test', 'val']
        self.set_type = {'train': 0, 'val': 1, 'test': 2}[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq
        self.step_size = step_size

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

    def __read_data__(self):
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path, self.data_path))
        
        cols_data = [col for col in df_raw.columns if col != 'date']
        df_data = df_raw[cols_data]
        
        num_train = int(len(df_raw) * 0.6)
        num_test = int(len(df_raw) * 0.2)
        border1s = [0, num_train - self.seq_len, len(df_raw) - num_test - self.seq_len]
        border2s = [num_train, num_train + (len(df_raw) - num_train - num_test), len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]
        
        if self.features == 'M' or self.features == 'MS':
            pass
        elif self.features == 'S':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        
        
        if self.scale:
            train_data = df_data.iloc[border1s[0]:border2s[0]].values
            self.scaler.fit(train_data)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values
        
        df_stamp = df_raw[['date']].iloc[border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp['date'])
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp['date'].dt.month
            df_stamp['day'] = df_stamp['date'].dt.day
            df_stamp['weekday'] = df_stamp['date'].dt.weekday
            df_stamp['hour'] = df_stamp['date'].dt.hour
            data_stamp = df_stamp.drop(['date'], axis=1).values
        elif self.timeenc == 1:
            # Example for time_features function
            data_stamp = self.time_features(df_stamp['date'], freq=self.freq)

        self.data_x = data[border1:border2 - self.pred_len]
        self.data_y = data[border1 + self.seq_len:border2]
        self.data_stamp = data_stamp

    def __getitem__(self, index):
        # Calculate the actual start index based on the step size
        actual_start_index = index * self.step_size

        seq_x = self.data_x[actual_start_index:actual_start_index + self.seq_len]
        seq_y = self.data_y[actual_start_index:actual_start_index + self.pred_len]
        seq_x_mark = self.data_stamp[actual_start_index:actual_start_index + self.seq_len]
        seq_y_mark = self.data_stamp[actual_start_index:actual_start_index + self.pred_len]

        return torch.tensor(seq_x,  dtype=torch.float32), torch.tensor(seq_y,  dtype=torch.float32), torch.tensor(seq_x_mark,  dtype=torch.float32), torch.tensor(seq_y_mark,  dtype=torch.float32)

    def __len__(self):
        # Adjust the total length to account for the step size
        total_steps = (len(self.data_x) - self.seq_len - self.pred_len + 1) // self.step_size
        return total_steps


    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)
# %%


class DilatedTCNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1, dropout_rate=0.2, dropout_=True, skip_=True):
        super(DilatedTCNBlock, self).__init__()
        
        # Calculate padding based on kernel size and dilation to maintain input length
        padding = (dilation * (kernel_size - 1)) // 2
        
        # First convolutional layer
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, dilation=dilation)
        self.norm1 = InstanceNorm1d(out_channels)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate) if dropout_ else nn.Identity()

        # Second convolutional layer
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=padding, dilation=dilation)
        self.norm2 = InstanceNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate) if dropout_ else nn.Identity()

        self.skip_ = skip_

    def forward(self, x):
        x_original = x.clone()
        
        # First conv -> norm -> relu -> dropout
        x = self.conv1(x)
        x = self.norm1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        
        # Second conv -> norm -> relu -> dropout
        x = self.conv2(x)
        x = self.norm2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        
        if self.skip_:
            x = x + x_original
        
        return x



# %%
# Take the new Positinal Encoding and Project layer from PatchTST
class PositionalEncoding2(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding2, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model%2 != 0:
            pe[:, 1::2] = torch.cos(position * div_term)[:,0:-1]
        else:
            pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
def PositionalEncoding(q_len, d_model, normalize=True):
    pe = torch.zeros(q_len, d_model)
    position = torch.arange(0, q_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
    
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    if normalize:
        pe = pe - pe.mean()
        pe = pe / (pe.std() * 10)
    return pe

def positional_encoding(pe, learn_pe, q_len, d_model):
    # Positional encoding
    if pe == None:
        W_pos = torch.empty((q_len, d_model)) # pe = None and learn_pe = False can be used to measure impact of pe
        nn.init.uniform_(W_pos, -0.02, 0.02)
        learn_pe = False
    elif pe == 'zero':
        W_pos = torch.empty((q_len, 1))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'zeros':
        W_pos = torch.empty((q_len, d_model))
        nn.init.uniform_(W_pos, -0.02, 0.02)
    elif pe == 'normal' or pe == 'gauss':
        W_pos = torch.zeros((q_len, 1))
        torch.nn.init.normal_(W_pos, mean=0.0, std=0.1)
    elif pe == 'uniform':
        W_pos = torch.zeros((q_len, 1))
        nn.init.uniform_(W_pos, a=0.0, b=0.1)
    elif pe == 'sincos': W_pos = PositionalEncoding(q_len, d_model, normalize=True)
    else: raise ValueError(f"{pe} is not a valid pe (positional encoder. Available types: 'gauss'=='normal', \
        'zeros', 'zero', uniform', 'lin1d', 'exp1d', 'lin2d', 'exp2d', 'sincos', None.)")
    return nn.Parameter(W_pos, requires_grad=learn_pe)

class ProjectedPositionalEncoding(nn.Module):
    def __init__(self, d_input, d_model, dropout=0.1, max_len=5000):
        super(ProjectedPositionalEncoding, self).__init__()
        self.d_input = d_input
        self.d_model = d_model
        self.dropout = nn.Dropout(p=dropout)

        # Projection layer: maps input dimension to model dimension
        self.projection = nn.Linear(d_input, d_model)

        #self.positionalencoding = PositionalEncoding(d_model,dropout, max_len = 5000)

    def forward(self, x):
        # Apply linear projection
        x_projected = self.projection(x)
        # Add positional encoding
        #x_pe = x_projected + self.positionalencoding(x)
        return x_projected

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.dropout = nn.Dropout(dropout)

        self.qkv_linear = nn.Linear(self.num_heads*d_model, d_model * 3)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
    

        # Transform and split Q, K, V into multiple heads
        qkv = self.qkv_linear(torch.cat([query, key, value], dim=-1))
    
        q, k, v = torch.split(qkv, qkv.size(-1) // 3, dim=-1)
        q, k, v = [x.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
                   for x in (q, k, v)]

        # Calculate attention using scaled dot product
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float('-inf'))
        p_attn = torch.softmax(scores, dim=-1)
        p_attn = self.dropout(p_attn)

        # Aggregate the heads
        x = torch.matmul(p_attn, v)
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.d_k)

        return self.out_linear(x), p_attn


class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=None, dropout=0.1, activation="relu"):
        super(EncoderLayer, self).__init__()
        d_ff = d_ff or 4 * d_model
        self.multihead_attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu if activation == "relu" else F.gelu

    def forward(self, x, mask=None):
        new_x, attn = self.multihead_attention(x, x, x, mask)
        x = x + self.dropout(new_x)
        y = x = self.norm1(x)
        y = self.dropout(self.activation(self.conv1(y.transpose(-1, 1))))
        y = self.dropout(self.conv2(y).transpose(-1, 1))
        return self.norm2(x + y), attn

class Encoder(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff=None, dropout=0.1, activation="relu"):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout, activation) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, mask=None):
        # x [B, L, D] - batch size, sequence length, dimensionality
        attns = []
        for layer in self.layers:
        
            x, attn = layer(x, mask)
            attns.append(attn)

        x = self.norm(x)
        return x, attns



# %%
# What removed: TCN_type, attention_type, general_skip, RevIN
class DWT_MLP_Model(nn.Module):
    def __init__(self, input_channels, seq_length, pred_length ,mlp_hidden_size, output_channels, decompose_layers=3, wave='haar', mode='symmetric', nhead=3, d_model=None, num_encoder_layers=3, dropout=0.1, dilation = 2, kernel_size = 3, dropout_ = True, skip_ = True,  pos_encoder_type_ = 'Projected'):
        super(DWT_MLP_Model, self).__init__()
        self.dwt_forward = DWT1DForward(J=decompose_layers, wave=wave, mode=mode)
        self.dwt_inverse = DWT1DInverse(wave=wave, mode=mode)
        self.seq_len = seq_length
        self.pred_len = pred_length
        self.dropout = dropout
        self.dilation = dilation
        self.kernel_size = kernel_size
        self.dropout_TF = dropout_ 
        self.skip_TF = skip_
        self.pos_encoder_type = pos_encoder_type_
        self.nhead = nhead
        
        if d_model is None:
            d_model = output_channels

        self.pos_dropout = nn.Dropout(self.dropout)
        # Assuming kernel_size can be used to derive kernel_set for illustration
        self.tcn_low = DilatedTCNBlock(input_channels, output_channels, dilation=self.dilation,kernel_size= self.kernel_size, dropout_rate= self.dropout, dropout_ = self.dropout_TF, skip_ = self.skip_TF)
        self.tcn_high_list = nn.ModuleList([DilatedTCNBlock(input_channels, output_channels, dropout_rate= self.dropout, kernel_size= self.kernel_size, dilation= self.dilation) for _ in range(decompose_layers)])


        self.revin_layer = RevIN(input_channels)
        
        # Positional Encoding
        if  self.pos_encoder_type == 'Projected':  
            self.pos_encoder = ProjectedPositionalEncoding(input_channels, d_model, max_len=5000)
            pe = 'zeros'
            learn_pe = False
            
            self.W_pos_low =  positional_encoding(pe, learn_pe, self.seq_len//(decompose_layers*2), d_model)
            self.W_pos_high_list = [positional_encoding(pe, learn_pe, self.seq_len//(decompose_layers*2), d_model) for _ in range(1,decompose_layers+1)]
        else:
            self.pos_encoder = PositionalEncoding(self.seq_len, d_model)
            #self.pos_encoder = PositionalEncoding(d_model, max_len = 5000)
        # Custom Transformer Encoder setup
        
        #self.attention = MultiHeadAttention # Assuming attention is defined elsewhere
        #encoder_layers_low = [EncoderLayer( d_model, self.nhead, d_ff=mlp_hidden_size, dropout=self.dropout, activation="gelu") for _ in range(num_encoder_layers)]

        self.transformer_low = Encoder(2, d_model, self.nhead, d_ff=mlp_hidden_size)

        # Transformer Encoders for high-frequency components, using custom Encoder
        self.transformer_high_list = nn.ModuleList(
            [Encoder(2, d_model ,self.nhead, d_ff=mlp_hidden_size) 
             for _ in range(decompose_layers)])
        

    def forward(self, x):
        #x = x.permute(0, 2, 1)  # Adjust dimensions for DWT
        x = self.revin_layer(x, 'norm')
        x = x.permute(0,2,1)

        
            
        x_low, x_highs = self.dwt_forward(x)
        x_low_tcn = self.tcn_low(x_low)
        x_low_combined = x_low_tcn
        
        x_low_combined = x_low + x_low_combined
      
                
        x_low_combined = x_low_combined.permute(0,2,1)
    
        x_low_combined = self.pos_encoder(x_low_combined)
        
        x_low_combined = self.pos_dropout(x_low_combined + self.W_pos_low)
        #x_low_combined = x_low_combined.permute(0,2,1)
        
        x_low_combined, _ = self.transformer_low(x_low_combined) # Adjusted for custom encoder
        x_low_combined = x_low_combined.permute(0,2,1)
        x_low_combined = x_low + x_low_combined
        
        # Process high-frequency components
        x_highs_processed = []
        for i, x_high in enumerate(x_highs):
            x_high_tcn = self.tcn_high_list[i](x_high)
            x_high_combined = x_high_tcn
          
            x_high_combined = x_high + x_high_combined
          
            x_high_combined = x_high_combined.permute(0,2,1)
         
            x_high_combined = self.pos_encoder(x_high_combined)
            x_high_combined = self.pos_dropout(x_high_combined + self.W_pos_high_list[i])
            x_high_combined, _ = self.transformer_high_list[i](x_high_combined)  # Adjusted for custom encoder
            x_high_combined = x_high.permute(0,2,1) + x_high_combined
            x_highs_processed.append(x_high_combined.permute(0,2,1))
        
        # Reconstruct the signal and adjust dimensions
        pred_out = self.dwt_inverse((x_low_combined, x_highs_processed)).permute(0, 2, 1)
    
        pred_out = self.revin_layer(pred_out, 'denorm')
        
        #pred_out = pred_out[:, :, :-4] # Do not make predictions for meta features
        pred_out = pred_out[:, -self.pred_len:, :]


        return pred_out




# Assuming DWT_MLP_Model is defined elsewhere, along with the necessary imports
seq_ = 24*4
pred_ = 24*4
# Define hyperparameter combinations
dropout_enabled = True
skip_enabled = True
num_encoder_size = 1
pos_encoder_type = 'Projected'
mlp_hidden = None
k_size = 5
s_size = 8
decompose_layer = 1
bs = 64
mt = 'zero'
wt = 'haar'
dilat = 3
# Define the ranges for the hyperparameters
learning_rates = np.logspace(-3, -2, 100)  # Learning rates between 1e-3 and 1e-2
dropout_rates = np.linspace(0.0, 0.2, 100)  # Dropout rates between 0 and 0.5
weight_decays = np.logspace(-4, -3, 100)  # Weight decays between 1e-4 and 1e-3
indices = [0,1,2,3,4,5] #np.random.choice(range(100), size=3, replace=False)
input_length = [24*4, 512]

count = 0

for i in indices:
    if i < 3:
        seq_length = input_length[0]
    else:
        seq_length = input_length[1]
    #lrs = learning_rates[i]
    #dr = dropout_rates[i]
    #wd = weight_decays[i]
    lrs =0.0052230056036904522
    dr = 0.10146011891748014
    wd = 1.0059977697794999e-04
                                            
    # Specify the file path
    root_path = '/home/choi/Wave_Transformer/optuna_/electricity/'
    data_path = 'electricity.csv'
    # Size parameters

    seq_len = seq_length # 24*4*4
    pred_len = 24*4
    #batch_size = bs
    # Initialize the custom dataset for training, validation, and testing
    train_dataset = Dataset_Custom(root_path=root_path, features= 'M', flag='train', data_path=data_path, step_size =s_size)
    val_dataset = Dataset_Custom(root_path=root_path, features= 'M',flag='val', data_path=data_path,step_size = s_size)
    test_dataset = Dataset_Custom(root_path=root_path, features= 'M',flag='test', data_path=data_path,step_size = s_size)

    # Optionally, initialize the dataset for prediction (if needed)
    #pred_dataset = Dataset_Pred(root_path=root_path, flag='pred', size=size, data_path=data_path, inverse=True)

    # Example on how to create DataLoaders for PyTorch training (adjust batch_size as needed)
    batch_size = bs
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, drop_last = True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last = True)
    dropout_rate = dr if dropout_enabled else 0.0
    # Adjust the model instantiation to include all hyperparameters
    model = DWT_MLP_Model(input_channels=321, seq_length=seq_len, pred_length = pred_,mlp_hidden_size=mlp_hidden, 
                        output_channels=321, decompose_layers=decompose_layer, 
                        dropout=dropout_rate, dilation=dilat, 
                        mode=mt, wave=wt, kernel_size=k_size, 
                        num_encoder_layers=num_encoder_size, nhead=3, 
                        dropout_=dropout_enabled,
                        skip_=True, pos_encoder_type_ = pos_encoder_type)

    # Define criterion and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lrs, 
                        weight_decay=wd)

    train_losses = []
    val_losses = []

    # Early stopping parameters
    patience = 10
    best_val_loss = float('inf')
    patience_counter = 0

    num_epochs = 50

    # Start the timer
    start_time = time.time()

    best_model_path = f"best_model_haar200_{i}_{num_encoder_size}_{skip_enabled}_{pos_encoder_type}_{bs}_{decompose_layer}_{k_size}_{s_size}_{mlp_hidden}.pt"


    for epoch in range(num_epochs):
        model.train()

        train_loss = 0.0

        for seq_x, seq_y, seq_x_mark, seq_y_mark in train_loader:

            inputs = seq_x
        
            targets = seq_y

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)  # Assuming specific output handling
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for seq_x, seq_y, seq_x_mark, seq_y_mark in val_loader:
                inputs = seq_x
                targets = seq_y
                outputs = model(inputs)
                loss = criterion(outputs, targets)  # Same output handling assumption
                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)
            #print(f"New best model saved at {best_model_path}")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered")
            break

    end_time = time.time()
    total_time = end_time - start_time
    print(f'Total Model Running Time: {total_time:.2f} seconds')
    best_model = DWT_MLP_Model(input_channels=321, seq_length=seq_len, pred_length = pred_, mlp_hidden_size=mlp_hidden, 
    output_channels=321, decompose_layers=decompose_layer, dropout=dropout_rate, dilation=dilat, 
    mode=mt, wave=wt, kernel_size=k_size, 
    num_encoder_layers=num_encoder_size, nhead=3, 
    dropout_=dropout_enabled,
    skip_=True, pos_encoder_type_ = pos_encoder_type)
    best_model.load_state_dict(torch.load(best_model_path))
    # Evaluation on test data
    best_model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for seq_x, seq_y, seq_x_mark, seq_y_mark in test_loader:
            inputs =  seq_x
            targets =seq_y
            outputs = best_model(inputs)
            loss = criterion(outputs, targets)  # Assuming specific output handling
            test_loss += loss.item() * inputs.size(0)

    test_loss /= len(test_loader.dataset)
    print(f'The {count}th model done.')
    count += 1
    print(f'Test Loss for configuration for Neu_Multihead: , {i}_{num_encoder_size}_{skip_enabled}_{pos_encoder_type}_{bs}_{decompose_layer}_{k_size}_{s_size}_{mlp_hidden}: {test_loss:.4f}')

# %%





KeyboardInterrupt: 